In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install folium
import folium
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 1000)
pd.set_option('max_rows', 30)
pd.set_option('max_columns', 1000)

# Data engineering
Testing API and reading data as pickle to optimize time 

In [2]:
#Testing API connection
import urllib.request as req
url = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=d543a7b1-f8cb-439f-8a5c-e56c5479eeb5&limit=1000000'  
fileobj = req.urlopen(url)
print (fileobj.read)

<bound method HTTPResponse.read of <http.client.HTTPResponse object at 0x000001DF43FD27F0>>


In [3]:
type(fileobj)

http.client.HTTPResponse

In [5]:
response = json.loads(fileobj.read())

In [32]:
result = response.get('result')
records = result.get('records')

In [52]:
df = pd.DataFrame(records)
#Changing to csv file because of API capacity
#Reading as pickle
df = pd.read_pickle('text.pkl')
df.head(2)

,id,idCarpeta,Ano_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,Ano_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,AlcaldiaHechos,ColoniaHechos,Calle_hechos,Calle_hechos2,latitud,longitud
0,1,8324429,2019,Enero,2019-04-01T00:00:00,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,FUERO COMUN,2018.0,Agosto,29/08/2018,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN,INSUGENTES SUR,NaN,19.36125,-99.18314
1,2,8324430,2019,Enero,2019-04-01T00:00:00,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCIÓN, ALTERACIÓN O FALSIFICACIÓN DE TÍTULOS AL PORTADOR, DOCUMENTOS DE CRÉDITO PÚBLICOS O VALES DE CANJE",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2018.0,Diciembre,15/12/2018,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,AV. CUATLAHUAC,NaN,19.47181,-99.16458


In [53]:
print(df.columns)
print(df.info())

Index(['id', 'idCarpeta', 'Ano_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Ano_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'AlcaldiaHechos', 'ColoniaHechos', 'Calle_hechos',
       'Calle_hechos2', 'latitud', 'longitud'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514048 entries, 0 to 514047
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               514048 non-null  int64  
 1   idCarpeta        514048 non-null  int64  
 2   Ano_inicio       514048 non-null  int64  
 3   Mes_inicio       514048 non-null  object 
 4   FechaInicio      514048 non-null  object 
 5   Delito           514048 non-null  object 
 6   Categoria        514048 non-null  object 
 7   Sexo             409002 non-null  object 
 8   Edad             323320 non-null  float64
 9   T

In [57]:
df.Ano_hecho.value_counts()

2019.0    257810
2020.0    205322
2021.0     27547
2018.0     16212
2017.0      2547
           ...  
1964.0         1
1962.0         1
1955.0         1
1952.0         1
1975.0         1
Name: Ano_hecho, Length: 58, dtype: int64

In [56]:
df.Categoria.value_counts()

DELITO DE BAJO IMPACTO                                     404018
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        29693
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        25583
ROBO A NEGOCIO CON VIOLENCIA                                18772
HECHO NO DELICTIVO                                           8272
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        7326
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        4122
HOMICIDIO DOLOSO                                             3796
VIOLACIÓN                                                    3529
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                3152
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      1974
ROBO A CASA HABITACIÓN CON VIOLENCIA                         1604
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA      1155
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 636
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      314
SECUESTRO 

In [59]:
df.fillna(0, inplace = True)
df_2019 = df[df.Ano_hecho == 2019]
df_2020 = df[df.Ano_hecho == 2020]
df_2021 = df[df.Ano_hecho == 2021]

In [62]:
df_2021.shape

(27547, 23)

# First Map Using 2021